In [15]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import torchvision
from torchvision.datasets import MNIST
import torchvision.transforms as transforms
import torch.nn.functional as F
import torch.nn as nn 

In [16]:
a=[0,1,2,3,4,5]
b=[34,23,34,45,56,45]

print(f'\n zip converts list to tuple ')
for x in zip(a, b):
    print(x,type(x))

print(f'\n zip converts list to int by one by')
for x,y in zip(a, b):
    print(x,type(x))

print(f'\n zip slice and converts list to tuple')
for x in zip(a,a[1:3]):
    print(x,type(x))

print(f'\n zip slice and converts list to int')
for x,y in zip(a,a[1:3]):
    print(x,type(x))


 zip converts list to tuple 
(0, 34) <class 'tuple'>
(1, 23) <class 'tuple'>
(2, 34) <class 'tuple'>
(3, 45) <class 'tuple'>
(4, 56) <class 'tuple'>
(5, 45) <class 'tuple'>

 zip converts list to int by one by
0 <class 'int'>
1 <class 'int'>
2 <class 'int'>
3 <class 'int'>
4 <class 'int'>
5 <class 'int'>

 zip slice and converts list to tuple
(0, 1) <class 'tuple'>
(1, 2) <class 'tuple'>

 zip slice and converts list to int
0 <class 'int'>
1 <class 'int'>


In [17]:
#Broadcast


In [18]:
#list Comprehension
x = [1,2,3,4,5,6,7,8,9]
x=np.array(x)
x[x>=4]

# ---------
list = [i for i in range(11) if i % 2 == 0]
print(list)

# ---------

matrix = [] 
for i in range(3):
    # Append an empty sublist inside the list
    matrix.append([])
    for j in range(5):
        matrix[i].append(j)
print(matrix)

# Nested list comprehension
matrix = [[j for j in range(5)] for i in range(3)]
print(matrix)

[0, 2, 4, 6, 8, 10]
[[0, 1, 2, 3, 4], [0, 1, 2, 3, 4], [0, 1, 2, 3, 4]]
[[0, 1, 2, 3, 4], [0, 1, 2, 3, 4], [0, 1, 2, 3, 4]]


In [ ]:
#iterable object
nums=[1,2,3,4,5,6]  #list object is an iterableble but not iterator
print(dir(nums))
nums=nums.__iter__() # or iter(nums)
print(f'\n', dir(nums))

for i in range(5,10):
    print(i)

# list of vowels
vowels = ["a", "e", "i", "o", "u"]
# iter() with a list of vowels
vowels_iter = iter(vowels)
print(next(vowels_iter))
print(next(vowels_iter))


In [20]:
#Module List  & Sequential 
'''
The main difference between Sequential is that ModuleList have not a forward method so the inner layers are not connected. 
Assuming we need each output of each layer in the decoder, we can store it by:
'''

#Module List
sizes=[1,16,32,32,64,128,64,32,32,1]
layers = nn.ModuleList([nn.Linear(in_f, out_f) for in_f, out_f in zip(sizes, sizes[1:])])
layers

#Sequential #1

import torch.nn.functional as F

class MyCNNClassifier(nn.Module):
    def __init__(self, in_c, n_classes):
        super().__init__()
        self.conv1 = nn.Conv2d(in_c, 32, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(32)

        self.fc1 = nn.Linear(32 * 28 * 28, 1024)
        self.fc2 = nn.Linear(1024, n_classes)
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = F.relu(x)
        
        x = self.conv2(x)
        x = self.bn2(x)
        x = F.relu(x)

        x = x.view(x.size(0), -1) # flat
        
        x = self.fc1(x)
        x = F.sigmoid(x)
        x = self.fc2(x)
        
        return x

In [ ]:
#Sequential #2
def conv_block(in_f, out_f, *args, **kwargs):
    return nn.Sequential(
        nn.Conv2d(in_f, out_f, *args, **kwargs),
        nn.BatchNorm2d(out_f),
        nn.ReLU()
    )

class MyCNNClassifier(nn.Module):
    def __init__(self, in_c, n_classes):
        super().__init__()
        #self.conv_block1 = conv_block(in_c, 32, kernel_size=3, padding=1)
        #self.conv_block2 = conv_block(32, 64, kernel_size=3, padding=1)
        self.encoder = nn.Sequential(
            conv_block(in_c, 32, kernel_size=3, padding=1),
            conv_block(32, 64, kernel_size=3, padding=1)
        )

        self.decoder = nn.Sequential(
            nn.Linear(32 * 28 * 28, 1024),
            nn.Sigmoid(),
            nn.Linear(1024, n_classes)
        )

    def forward(self, x):
        x = self.encoder(x)
        x = x.view(x.size(0), -1) # flat
        x = self.decoder(x)
        return x


model = MyCNNClassifier(1, 10)
print(model)

In [ ]:
#Sequential #3
def conv_block(in_f, out_f, *args, **kwargs):
    return nn.Sequential(
        nn.Conv2d(in_f, out_f, *args, **kwargs),
        nn.BatchNorm2d(out_f),
        nn.ReLU()
    )
def dec_block(in_f, out_f):
    return nn.Sequential(
        nn.Linear(in_f, out_f),
        nn.Sigmoid()
    )
#Using * in front of a list expands out the members as individual arguments. So, the following two function calls will be equivalent:
class MyCNNClassifier(nn.Module):
    def __init__(self, in_c, enc_sizes, dec_sizes,  n_classes):
        super().__init__()
        self.enc_sizes = [in_c, *enc_sizes]  
        self.dec_sizes = [32 * 28 * 28, *dec_sizes]

        conv_blokcs = [conv_block(in_f, out_f, kernel_size=3, padding=1) 
                       for in_f, out_f in zip(self.enc_sizes, self.enc_sizes[1:])]
        
        self.encoder = nn.Sequential(*conv_blokcs)
        
        dec_blocks = [dec_block(in_f, out_f) 
                       for in_f, out_f in zip(self.dec_sizes, self.dec_sizes[1:])]

        self.decoder = nn.Sequential(*dec_blocks)

        self.last = nn.Linear(self.dec_sizes[-1], n_classes)

        
    def forward(self, x):
        x = self.encoder(x)
        x = x.view(x.size(0), -1) # flat
        x = self.decoder(x)
        return x
        
model = MyCNNClassifier(1, [32,64], [1024, 512], 10)
print(model)